# 📊 Final Project - Customer Churn Prediction for Sendo Farm

## Problem Definition

Sendo Farm is an online grocery e-commerce platform that delivers essential goods, similar to a supermarket, directly to consumers for their daily meals.  
Because the products are groceries—especially fresh items such as meat, fish, fruits, and vegetables—customer satisfaction is highly sensitive to quality.  

Some customers who are dissatisfied with product quality or after-sales service choose to file complaints, request refunds, or report missing items. However, many others leave silently without expressing dissatisfaction and never purchase from Sendo Farm again.  

Customer churn is costly, especially in the grocery business where customers make frequent and recurring purchases.  
Thus, the goal of this project is to build a **supervised machine learning model** that predicts which customers are at risk of churning. This will allow Sendo Farm to proactively take preventive actions, such as personalized campaigns, compensation, or loyalty offers, to improve customer experience and retention.  

### Dataset Description
The available data includes:
- **Customer infomation** (customerid, monthly_frequency, recency, days_stop_frequency, total_value, last_rating, avarage_rating, last_complain, last_refund, total_comlains, total_refunds) 
- **Transaction history** (customerid, orderid, orderdate,rating, is_compalained, order_value).
     

These features will be engineered into customer-level data suitable for supervised ML classification (churn vs non-churn).


In [5]:
# 📦 Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

# Set display options for pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Matplotlib is building the font cache; this may take a moment.


## 1. Data Loading & Initial Merge

### Overview
This section focuses on loading and reviewing transaction data from the grocery_transactions.csv file, specifically targeting transactions from **June, July, and August 2025** for **10,000 customers**. The primary objectives are:

1. **Load the dataset** and perform initial data exploration
2. **Review data quality** and structure

3. **The printed out information as below:**

Total rows (transactions): 77,618

Unique customer IDs: 10,000

Data columns (total 9 columns):so.ordernumber, orderdate, order_value, customerid, applyclaimdate, refundguid, reasoncancelcode, rating, is_fresh



In [15]:
# 📊 Load grocery transactions data
df = pd.read_csv('../data/grocery_transactions.csv')

# Display basic information about the dataset
print(f"Total rows (transactions): {len(df):,}")
#Print unique customerid here 
print(f"Unique customer IDs: {df['customerid'].nunique():,}")
# Display column information
print("Column Information:")
print(df.info())


Total rows (transactions): 77,618
Unique customer IDs: 10,000
Column Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77618 entries, 0 to 77617
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   so.ordernumber    77618 non-null  int64  
 1   orderdate         77618 non-null  object 
 2   order_value       77618 non-null  float64
 3   customerid        77618 non-null  int64  
 4   applyclaimdate    560 non-null    object 
 5   refundguid        1161 non-null   object 
 6   reasoncancelcode  4475 non-null   object 
 7   rating            10261 non-null  float64
 8   is_fresh          77618 non-null  int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 5.3+ MB
None


## 2. Preliminary Feature Engineering

Before diving deep into analysis, we'll create some essential customer-level features by aggregating the merged data. This gives us a basic dataset for initial exploration.

### Feature Aggregation
- **customerid**: Customer level group by (10K)
- **total_order**: Total number of orders per customer (count from ordernumber).
- **total_spend**: Sum of money spent by each customer (sum from order_value).
- **is_bad_rating**: Normalize rating given by the customer (If average rating < 3.5 then 1 else 0).
- **is_bad_last_rating**: Normalize last rating given by the customer (If most recent rating < 3.5 then 1 else 0).
- **total_incidents**: Total number of incidents (refunds, incidents) by a customer (count(refundguid) + count(applyclaimdate) ).
- **is_incident_last_order**: Was there an incident at last order (If last order is incident or refurnd then 1 else 0).
- **is_fresh_last_order**: Is there any fresh item in last order (If last order having is_fresh = true then 1 else 0). 
- **is_canceled_last order** Was the last order  cancelled? (If reasoncancelcode of last order is NOT NULL then 1 else 0 ).
- **Recency**: Number of days since the customer's last order ( Day('08-31-2925' - max(orderdate))).
- **is_churn**: This is the label column (If Recency > 15 then 1 else 0).

Printed out the head and dataset info.

In [17]:
# 🔧 Create customer-level features by aggregating transaction data
print("Creating customer-level features...")
print("=" * 50)

# Ensure orderdate is datetime
df['orderdate'] = pd.to_datetime(df['orderdate'])

# Group by customerid and create aggregated features
customer_features = df.groupby('customerid').agg({
    'so.ordernumber': 'count',  # total_order
    'order_value': 'sum',       # total_spend
    'rating': 'last',           # last_rating only
    'refundguid': 'count',      # count refunds
    'applyclaimdate': 'count',  # count claims
    'reasoncancelcode': 'count', # count cancellations
    'is_fresh': 'last',         # is_fresh_last_order
    'orderdate': 'max'          # last_order_date
}).reset_index()

# Flatten column names
customer_features.columns = [
    'customerid', 'total_order', 'total_spend', 'last_rating',
    'total_refunds', 'total_claims', 'total_cancellations', 'is_fresh_last_order', 'last_order_date'
]

# Calculate avg_rating separately - only for orders with actual ratings (not NULL)
print("Calculating average rating for orders with actual ratings...")
avg_rating_by_customer = df[df['rating'].notna()].groupby('customerid')['rating'].mean()
customer_features['avg_rating'] = customer_features['customerid'].map(avg_rating_by_customer)

# Calculate additional features
print("Calculating derived features...")

# 1. is_bad_rating: If average rating < 3.5 then 1 else 0 (only for customers with ratings)
customer_features['is_bad_rating'] = (customer_features['avg_rating'] < 3.5).astype(int)
# For customers with no ratings, is_bad_rating = 0 (neutral)
customer_features['is_bad_rating'] = customer_features['is_bad_rating'].fillna(0)

# 2. is_bad_last_rating: If most recent rating < 3.5 then 1 else 0
customer_features['is_bad_last_rating'] = (customer_features['last_rating'] < 3.5).astype(int)

# 3. total_incidents: Total number of incidents (refunds + claims)
customer_features['total_incidents'] = customer_features['total_refunds'] + customer_features['total_claims']

# 4. is_incident_last_order: Was there an incident at last order
# We need to check if the last order had an incident or refund
last_orders = df.sort_values('orderdate').groupby('customerid').tail(1)
last_orders_incidents = last_orders.set_index('customerid')[['refundguid', 'applyclaimdate']].notna().any(axis=1)
customer_features['is_incident_last_order'] = customer_features['customerid'].map(last_orders_incidents).fillna(0).astype(int)

# 5. is_canceled_last_order: Was the last order cancelled?
last_orders_canceled = last_orders.set_index('customerid')['reasoncancelcode'].notna()
customer_features['is_canceled_last_order'] = customer_features['customerid'].map(last_orders_canceled).fillna(0).astype(int)

# 6. Recency: Number of days since the customer's last order
reference_date = pd.to_datetime('2025-08-31')  # End of August 2025
customer_features['recency'] = (reference_date - customer_features['last_order_date']).dt.days

# 7. is_churn: If Recency > 15 then 1 else 0
customer_features['is_churn'] = (customer_features['recency'] > 15).astype(int)

# Handle missing values
customer_features['last_rating'] = customer_features['last_rating'].fillna(0)
# avg_rating remains NaN for customers with no ratings (this is correct behavior)

print("✅ Customer-level features created successfully!")
print(f"📊 Dataset shape: {customer_features.shape}")
print()

# Display basic information
print("🔍 Customer Features Dataset Information:")
print("=" * 50)
print(customer_features.info())
print()

# Display first few rows
print("👀 First 10 rows of customer features:")
print("=" * 50)
print(customer_features.head(10))
print()

# Display basic statistics
print("📊 Basic Statistics:")
print("=" * 50)
print(customer_features.describe())
print()

# Display churn distribution
print("🎯 Churn Distribution:")
print("=" * 50)
churn_counts = customer_features['is_churn'].value_counts()
print(f"Non-churn customers (0): {churn_counts[0]:,} ({churn_counts[0]/len(customer_features)*100:.1f}%)")
print(f"Churn customers (1): {churn_counts[1]:,} ({churn_counts[1]/len(customer_features)*100:.1f}%)")
print()

# Display feature summary
print("📈 Feature Summary:")
print("=" * 50)
print(f"• Total customers: {len(customer_features):,}")
print(f"• Average orders per customer: {customer_features['total_order'].mean():.2f}")
print(f"• Average spend per customer: ${customer_features['total_spend'].mean():.2f}")
print(f"• Average rating (customers with ratings): {customer_features['avg_rating'].mean():.2f}")
print(f"• Customers with ratings: {customer_features['avg_rating'].notna().sum():,}")
print(f"• Customers with bad ratings: {customer_features['is_bad_rating'].sum():,}")
print(f"• Customers with incidents: {customer_features['total_incidents'].sum():,}")
print(f"• Average recency: {customer_features['recency'].mean():.1f} days")

print("\n✅ Preliminary feature engineering completed!")

Creating customer-level features...
Calculating average rating for orders with actual ratings...
Calculating derived features...
✅ Customer-level features created successfully!
📊 Dataset shape: (10000, 17)

🔍 Customer Features Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   customerid              10000 non-null  int64         
 1   total_order             10000 non-null  int64         
 2   total_spend             10000 non-null  float64       
 3   last_rating             10000 non-null  float64       
 4   total_refunds           10000 non-null  int64         
 5   total_claims            10000 non-null  int64         
 6   total_cancellations     10000 non-null  int64         
 7   is_fresh_last_order     10000 non-null  int64         
 8   last_order_date         10000 non-null 

## 3. Targeted Exploratory Data Analysis (EDA)

With our preliminary features, we can now conduct a targeted EDA to understand the relationship between these features and customer churn. The goal is to identify potential churn drivers and generate hypotheses for more advanced feature engineering.

In [ ]:
# 🔹 Define Churn based on Recency
# Assuming a simple rule: churn if a customer has not ordered in the last 30 days.
customer_features['churn'] = (customer_features['days_since_last_order'] > 30).astype(int)

# Check Churn Distribution
churn_dist = customer_features['churn'].value_counts(normalize=True)
print("\nChurn Distribution:")
print(churn_dist)
churn_dist.plot(kind='bar', title='Churn vs Non-Churn Distribution')
plt.show()

# Analyze Churn Rate vs Key Features
print("\nChurn rate by number of incidents:")
print(customer_features.groupby('num_incidents')['churn'].mean())

print("\nChurn rate by average rating:")
bins = [1, 2, 3, 4, 5]
customer_features['avg_rating_bin'] = pd.cut(customer_features['avg_rating'], bins=bins, labels=False)
print(customer_features.groupby('avg_rating_bin')['churn'].mean())

# Visualize relationships
sns.boxplot(x='churn', y='days_since_last_order', data=customer_features)
plt.title('Days Since Last Order vs Churn')
plt.show()

sns.barplot(x='num_incidents', y='churn', data=customer_features, estimator=np.mean)
plt.title('Churn Rate vs Number of Incidents')
plt.show()

## 4. Advanced Feature Engineering

Based on the EDA findings, we will now engineer more sophisticated features that capture behavioral patterns over time, such as changes in order frequency. These features are often more predictive of churn.

In [ ]:
# 🔹 Feature: Order Frequency Change
# This feature captures a sudden drop in a customer's purchasing frequency.

# Sort data by customer and date for time-series analysis
orders_sorted = orders.sort_values(by=['customer_id', 'order_date'])

# Calculate rolling order frequency (e.g., last 30 days)
orders_sorted['rolling_30d_freq'] = orders_sorted.groupby('customer_id')['order_date'].rolling('30D').count().reset_index(level=0, drop=True)

# Calculate order frequency for specific periods (e.g., month -1, month -2)
current_month = orders_sorted['order_date'].max().to_period('M')

# Filter orders from the last two months
orders_last_2_months = orders_sorted[orders_sorted['order_date'] >= (current_month - 1).to_timestamp()]

monthly_orders = orders_last_2_months.groupby(['customer_id', orders_last_2_months['order_date'].dt.to_period('M')]).size().unstack(fill_value=0)

# Create frequency change feature
if monthly_orders.shape[1] >= 2:
    monthly_orders['freq_month_minus1'] = monthly_orders.iloc[:, -1]
    monthly_orders['freq_month_minus2'] = monthly_orders.iloc[:, -2]
    monthly_orders['freq_change'] = monthly_orders['freq_month_minus1'] - monthly_orders['freq_month_minus2']
    customer_features = customer_features.merge(monthly_orders[['freq_change']], on='customer_id', how='left')
else:
    print("Not enough data to create frequency change feature.")
    customer_features['freq_change'] = 0
    
print("\nUpdated Customer Features with Advanced Features:")
print(customer_features.head())

## 5. Model Building & Training

Now we have a rich feature set, we can train and evaluate different supervised machine learning models to predict churn.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, XGBClassifier

# Handle potential missing values from merge
customer_features.fillna(0, inplace=True)

# Select features and target
feature_cols = ['total_orders', 'total_spend', 'avg_rating', 'num_incidents', 'days_since_last_order', 'freq_change']

X = customer_features[feature_cols]
y = customer_features['churn']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Train models
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"\n{name} - Classification Report:")
    print(classification_report(y_test, y_pred))
    y_proba = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_proba)
    print(f"{name} - ROC AUC: {auc:.4f}")

## 6. Results & Evaluation
- Compare Accuracy, F1-score, ROC-AUC  
- Confusion matrix  
- ROC curve  

In [ ]:
models = {"LR": lr, "RF": rf, "XGB": xgb}

for name, model in models.items():
    y_pred = model.predict(X_test)
    print(f"\n{name} - Classification Report")
    print(classification_report(y_test, y_pred))
    
    # ROC
    y_proba = model.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test, y_proba)
    print(f"{name} - ROC AUC: {auc:.4f}")

In [ ]:
# Confusion matrix for best model (e.g. XGB)
cm = confusion_matrix(y_test, y_pred_xgb)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix - XGBoost")
plt.show()

## 6. Conclusion & Future Work

Based on the evaluation metrics, we can determine the best-performing model. This project serves as a strong foundation, and future improvements could include:

- **Feature Scaling**: Apply `StandardScaler` to numerical features for models like Logistic Regression.
- **Hyperparameter Tuning**: Use `GridSearchCV` or `RandomizedSearchCV` to find optimal parameters for models like Random Forest and XGBoost.
- **Advanced Features**: Incorporate more features from the `Products` table (e.g., proportion of fresh vs. dry goods purchased).
- **Deployment**: Integrate the best model into a real-time system to identify at-risk customers dynamically.